In [1]:
# Predict Whether Email is spam or not using SVM /Naive Bayes

In [2]:
from pyspark.mllib.classification import   SVMWithSGD, NaiveBayes
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint


In [3]:
  
## Each line has text from one Email
spam = sc.textFile("/FileStore/tables/spam")
ham = sc.textFile("/FileStore/tables/ham")

In [4]:
spam_words = spam.map(lambda email: email.split())
ham_words = ham.map(lambda email: email.split())

print(spam_words.take(1))
print(ham_words.take(1))

[['You', 'have', '1', 'new', 'message.', 'Please', 'call', '08712400200.']]
[['Rofl.', 'Its', 'true', 'to', 'its', 'name']]

In [5]:
# Create a HashingTF instance to map email text to vectors of features.
tf = HashingTF(numFeatures = 200)
spam_features = tf.transform(spam_words)
ham_features = tf.transform(ham_words)

print(spam_features.take(1))
print(ham_features.take(1))

[SparseVector(200, {22: 2.0, 59: 1.0, 82: 1.0, 95: 1.0, 100: 1.0, 106: 1.0, 125: 1.0})]
[SparseVector(200, {8: 1.0, 16: 1.0, 28: 1.0, 63: 1.0, 70: 1.0, 165: 1.0})]

In [6]:
#Create LabeledPoint datasets for positive (spam) and negative (ham) examples.¶
spam_samples = spam_features.map(lambda features:LabeledPoint(1, features))
ham_samples = ham_features.map(lambda features:LabeledPoint(0, features))

print(spam_samples.take(1))
print(ham_samples.take(1))

[LabeledPoint(1.0, (200,[22,59,82,95,100,106,125],[2.0,1.0,1.0,1.0,1.0,1.0,1.0]))]
[LabeledPoint(0.0, (200,[8,16,28,63,70,165],[1.0,1.0,1.0,1.0,1.0,1.0]))]

In [7]:
# Split the data set 80/20
samples = spam_samples.union(ham_samples)
[training_data, test_data] = samples.randomSplit([0.8, 0.2])
training_data.cache()
test_data.cache()

Out[ 16 ]: PythonRDD[24] at RDD at PythonRDD.scala:50

In [8]:
def score(model):
    predictions = model.predict(test_data.map(lambda x: x.features))
    labels_and_preds = test_data.map(lambda x: x.label).zip(predictions)
    accuracy = labels_and_preds.filter(lambda x: x[0] == x[1]).count() / float(test_data.count())
    return accuracy

In [9]:
#Train a SVM model¶
algo = SVMWithSGD()
model = algo.train(training_data)
score(model)

Out[ 18 ]: 0.8808243727598566

In [10]:
# Train a Naive Bayes model
algo = NaiveBayes()
model = algo.train(training_data)

In [11]:
score(model)

Out[ 22 ]: 0.9256272401433692